In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.functions import year
from pyspark.sql.functions import expr
from pyspark.sql.functions import col

import matplotlib.pyplot as plt
import pandas as pd
import time

In [20]:
# Armazenando o tempo de início
start_time = time.time()

In [21]:
# Iniciando uma sessão do Spark
spark = SparkSession.builder.appName("Temperatura_Media").getOrCreate()

In [22]:
# Carregando o arquivo CSV como um DataFrame do Spark
dados = spark.read.csv("GlobalLandTemperaturesByCity.csv", header=True, sep=",")

In [23]:
# Verificando o formato dos dados
dados.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [24]:
# Removendo linhas com valores ausentes
dados = dados.na.drop()

In [25]:
dados.show(10)

+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01| 14.050999999999998|                        1.347|Århus|Denmark|  57.05N|   10.33E|
|1744-07-01|             16.082|                        1.396|Århus|Denmark|  57.05N|   10.33E|
|1744-09-01| 12.780999999999999|                        1.454|Århus|Denmark|  57.05N|   10.33E|
|1744-10-01|               7.95|                         1.63|Århus|Denmark|  57.05N|   10.33E|
|1744-11-01|  4.638999999999999|        

In [26]:
# Convertendo a coluna "dt" para dados
dados = dados.withColumn("dt", to_date(dados["dt"], "yyyy-MM-dd"))

In [27]:
# Passando a coluna AverageTemperature para dados numéricos
from pyspark.sql.functions import col

try:
    dados = dados.withColumn("AverageTemperature_numeric", col("AverageTemperature").cast("double"))
except:
    print("col is not numeric")

dados.show()

+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|AverageTemperature_numeric|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+--------------------------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|                     6.068|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|        5.7879999999999985|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|                    10.644|
|1744-06-01| 14.050999999999998|                        1.347|Århus|Denmark|  57.05N|   10.33E|        14.050999999999998|
|1744-07-01|             16.082|                        1.396|Århus|Denmark|  57.05N|   10.33E|                    16.082|
|1744-09-01| 12.

In [28]:
# Extraindo o ano da coluna "dt"
dados = dados.withColumn("year", year(dados["dt"]))

In [29]:
# Agrupando as temperaturas por ano e cidade
dados_agrupados = dados.groupBy(["year", "City"]).mean("AverageTemperature_numeric")
dados_agrupados.show(5)

+----+-------+-------------------------------+
|year|   City|avg(AverageTemperature_numeric)|
+----+-------+-------------------------------+
|1767|  Århus|              6.946416666666668|
|1781|  Århus|              8.528916666666667|
|1766|  Çorlu|                       13.39575|
|1868|  Çorlu|                        13.0665|
|1938|Öskemen|             1.3139999999999992|
+----+-------+-------------------------------+
only showing top 5 rows



In [ ]:
# Obtendo as cidades únicas
cidades = dados.select("City").distinct().rdd.flatMap(lambda x: x).collect()
print(cidades)


['Antwerp', 'Araruama', 'Bangalore', 'Benxi', 'Cajamarca', 'Cicalengka', 'Copiapo', 'Da Nang', 'Dos Hermanas', 'Bryansk', 'Charleston', 'Chikmagalur', 'Chisinau', 'Corona', 'Abuja', 'Acapulco', 'Arequipa', 'Asahikawa', 'Badajoz', 'Bahawalnagar', 'Barbacena', 'Bojnurd', 'Brasov', 'Bukit Mertajam', 'Cà Mau', 'Cairo', 'Casablanca', 'Dimapur', 'Americana', 'Bhind', 'Bohicon', 'Cúa', 'Chavakachcheri', 'Cileunyi', 'Agadir', 'Ashikaga', 'Aurangabad', 'Bauchi', 'Brunswick', 'Cairns', 'Colatina', 'Ciego De Ávila', 'Bissau', 'Brighton', 'Belgorod', 'Bhubaneswar', 'Chenghai', 'Coral Springs', 'Alcorcón', 'Bolton', 'Århus', 'Anchorage', 'Araras', 'Bayamo', 'Birganj', 'Allentown', 'Asfi', 'Dourados', 'Edirne', 'Abohar', 'Anaheim', 'Cachoeirinha', 'Canberra', 'Chuncheng', 'Curitiba', 'Amersfoort', 'Carapicuíba', 'Coquimbo', 'Cumaná', 'Chattanooga', 'Edinburgh', 'Ajmer', 'Algeciras', 'Campo Grande', 'Concordia', 'Dallas', 'Adana', 'Batman', 'Dalian', 'Angeles', 'Bharuch', 'Bradford', 'Braga', 'Brakpa

In [ ]:
from pyspark.sql.functions import max, min

# Agrupando as temperaturas por cidade e calcular a temperatura máxima e mínima
dados_agrupados = dados.groupBy("City").agg(max("AverageTemperature_numeric").alias("max_temp"),
                                            min("AverageTemperature_numeric").alias("min_temp"))

# Calculando a diferença entre as temperaturas máxima e mínima
dados_diferenca = dados_agrupados.withColumn("diferenca_temp", expr("max_temp - min_temp"))

# Selecionando as colunas "City" e "diferenca_temp"
dados_final = dados_diferenca.select("City", "diferenca_temp")

# Exibindo os resultados
dados_final.show()


In [ ]:
# Ordenando o DataFrame em ordem crescente de acordo com a coluna "diferenca_temp"
dados_final = dados_final.sort(col("diferenca_temp").desc())

# Exibindo os resultados
dados_final.show()


In [33]:
# Criando um novo DataFrame com apenas as 10 primeiras linhas de "dados_final"
top_cities = dados_final.limit(10)

# Exibindo o novo DataFrame
top_cities.show()

+--------------+------------------+
|          City|    diferenca_temp|
+--------------+------------------+
|        Yichun|61.010000000000005|
|       Norilsk| 59.57599999999999|
|    Ust Ilimsk|59.022999999999996|
|         Kyzyl|            58.816|
|        Hailar|57.830999999999996|
|       Yakeshi|57.830999999999996|
|Nizhnevartovsk|56.254000000000005|
|          Nehe|             55.81|
|        Bratsk|55.733999999999995|
|         Heihe|            55.309|
+--------------+------------------+



In [34]:
# Armazenando o tempo de término
end_time = time.time()

# Calculando o tempo total de execução
execution_time = end_time - start_time

# Exibindo o tempo total de execução
print("Tempo de execução: {:.2f} segundos".format(execution_time))

Tempo de execução: 123.31 segundos
